In [1]:
import os
# os.chdir('../')

from ast import arg
from DeepMTP.simple_hyperband import BaseWorker
from DeepMTP.simple_hyperband import HyperBand
from DeepMTP.dataset import load_process_MLC, load_process_MTR, load_process_DP, process_dummy_MLC, process_dummy_MTR, process_dummy_DP, load_process_MC
from DeepMTP.utils.data_utils_v2 import data_process, BaseDataset
from DeepMTP.utils.utils import generate_config

import ConfigSpace.hyperparameters as CSH
import ConfigSpace as CS

/home/dimitriosi/.local/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# define the configuration space
cs= CS.ConfigurationSpace()

lr= CSH.UniformFloatHyperparameter(
    "learning_rate", lower=1e-6, upper=1e-3, default_value="1e-3", log=True
)
cs.add_hyperparameters([lr])

embedding_size= CSH.UniformIntegerHyperparameter(
    "embedding_size", lower=8, upper=2048, default_value=64, log=False
)

instance_branch_layers= CSH.UniformIntegerHyperparameter(
    "instance_branch_layers", lower=1, upper=2, default_value=1, log=False
)

instance_branch_nodes_per_layer= CSH.UniformIntegerHyperparameter(
    "instance_branch_nodes_per_layer", lower=8, upper=2048, default_value=64, log=False
)

target_branch_layers = CSH.UniformIntegerHyperparameter(
    "target_branch_layers", lower=1, upper=2, default_value=1, log=False
)

target_branch_nodes_per_layer = CSH.UniformIntegerHyperparameter(
    "target_branch_nodes_per_layer", lower=8, upper=2048, default_value=64, log=False
)

dropout_rate = CSH.UniformFloatHyperparameter(
    "dropout_rate", lower=0.0, upper=0.9, default_value=0.4, log=False
)

batch_norm = CSH.CategoricalHyperparameter("batch_norm", ["yes", "no"])

cs.add_hyperparameters(
    [
        embedding_size,
        instance_branch_layers,
        instance_branch_nodes_per_layer,
        target_branch_layers,
        target_branch_nodes_per_layer,
        dropout_rate,
        batch_norm,
    ]
)

cond = CS.GreaterThanCondition(dropout_rate, instance_branch_layers, 1)
# cs.add_condition(cond)

cond2 = CS.GreaterThanCondition(batch_norm, instance_branch_layers, 1)
# cs.add_condition(cond2)

cond3 = CS.GreaterThanCondition(dropout_rate, target_branch_layers, 1)
# cs.add_condition(cond)

cond4 = CS.GreaterThanCondition(batch_norm, target_branch_layers, 1)
# cs.add_condition(cond2)

cs.add_condition(CS.OrConjunction(cond, cond3))
cs.add_condition(CS.OrConjunction(cond2, cond4))

(batch_norm | instance_branch_layers > 1 || batch_norm | target_branch_layers > 1)

In [3]:
data = load_process_MLC(dataset_name='yeast', variant='undivided')
train, val, test, data_info = data_process(data, validation_setting='B', verbose=True)

Processing...
yeast:undivided - exists, not redownloading
Done
Interaction file: 2d numpy array format detected
Interaction file: checking format consistency... Passed
Interaction file: checking instance id format consistency... Passed
Interaction file: checking target id type consistency... Passed

Interaction file: checking target variable type consistency... Passed
Automatically detected type of target variable type: binary

-- Test set was not provided, could not detect if novel instances exist or not 
-- Test set was not provided, could not detect if novel targets exist or not 

Instance features file: processing features... Done

Cross input consistency for (numpy) interaction data and instance features checks out
-- Same instance ids in the interaction and features files for the train set

Splitting train to train-test according to validation setting B... Done
Splitting train to train-val according to validation setting B... Done


In [4]:
data_info

{'detected_validation_setting': 'B',
 'detected_problem_mode': 'classification',
 'instance_branch_input_dim': 103,
 'target_branch_input_dim': 14}

In [5]:
config = {    

    'hpo_results_path': './hyperband/',

    'instance_branch_input_dim': data_info['instance_branch_input_dim'],
    'target_branch_input_dim': data_info['target_branch_input_dim'],
    'validation_setting': data_info['detected_validation_setting'],
    'enable_dot_product_version': True,
    'problem_mode': data_info['detected_problem_mode'],

    'compute_mode': 'cuda:1',
    'train_batchsize': 512,
    'val_batchsize': 512,
    'num_epochs': 6,
    'num_workers': 8,

    'metrics': ['hamming_loss', 'auroc'],
    'metrics_average': ['macro'],
    'patience': 10,

    'evaluate_train': True,
    'evaluate_val': True,

    'verbose': True,
    'results_verbose': False,
    'use_early_stopping': True,
    'use_tensorboard_logger': True,
    'wandb_project_name': 'DeepMTP_v2',
    'wandb_project_entity': 'diliadis',
    'metric_to_optimize_early_stopping': 'loss',
    'metric_to_optimize_best_epoch_selection': 'loss',

    'instance_branch_architecture': 'MLP',

    'target_branch_architecture': 'MLP',

    'save_model': True,

    'eval_every_n_epochs': 10,

    'additional_info': {'eta': 3, 'max_budget': 9}
    }

In [6]:
worker = BaseWorker(
    train, val, test, data_info, config, 'loss'
)

In [7]:
worker.base_config

{'hpo_results_path': './hyperband/',
 'instance_branch_input_dim': 103,
 'target_branch_input_dim': 14,
 'validation_setting': 'B',
 'enable_dot_product_version': True,
 'problem_mode': 'classification',
 'compute_mode': 'cuda:1',
 'train_batchsize': 512,
 'val_batchsize': 512,
 'num_epochs': 6,
 'num_workers': 8,
 'metrics': ['hamming_loss', 'auroc'],
 'metrics_average': ['macro'],
 'patience': 10,
 'evaluate_train': True,
 'evaluate_val': True,
 'verbose': True,
 'results_verbose': False,
 'use_early_stopping': True,
 'use_tensorboard_logger': True,
 'wandb_project_name': 'DeepMTP_v2',
 'wandb_project_entity': 'diliadis',
 'metric_to_optimize_early_stopping': 'loss',
 'metric_to_optimize_best_epoch_selection': 'loss',
 'instance_branch_architecture': 'MLP',
 'target_branch_architecture': 'MLP',
 'save_model': True,
 'eval_every_n_epochs': 10,
 'additional_info': {'eta': 3, 'max_budget': 9}}

In [8]:
hb = HyperBand(
    base_worker=worker,
    configspace=cs,
    eta=config['additional_info']['eta'],
    max_budget=config['additional_info']['max_budget'],
    direction="min",
)

In [9]:
hb.run_optimizer()

Selected device: cuda:1
Starting training...


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: diliadis. Use `wandb login --relogin` to force relogin


Epoch:0... Done
  Validating... Calculating val performance... Done
Done
Starting testing... Calculating test performance... Done
Done


test_auroc_macro,▁
test_hamming_loss_macro,▁
train_auroc_macro,▁
train_hamming_loss_macro,▁
train_loss,▁
val_auroc_macro,▁
val_hamming_loss_macro,▁
val_loss,▁
test_auroc_macro,0.5084
test_hamming_loss_macro,0.69599
train_auroc_macro,0.50153


+-------+--------+---------+--------------------+-------------+
|  mode | #epoch |   loss  | hamming_loss_macro | auroc_macro |
+-------+--------+---------+--------------------+-------------+
| train |   0    | 69.7271 |       0.6972       |    0.5015   |
+-------+--------+---------+--------------------+-------------+
+------+--------+---------+--------------------+-------------+----------------+
| mode | #epoch |   loss  | hamming_loss_macro | auroc_macro | early_stopping |
+------+--------+---------+--------------------+-------------+----------------+
| val  |   0    | 10.1112 |       0.7025       |    0.4928   |      0/10      |
+------+--------+---------+--------------------+-------------+----------------+
+------+--------+------+--------------------+-------------+
| mode | #epoch | loss | hamming_loss_macro | auroc_macro |
+------+--------+------+--------------------+-------------+
| test |   0    |  -   |       0.696        |    0.5084   |
+------+--------+------+----------------

...

test_auroc_macro,▁
test_hamming_loss_macro,▁
train_auroc_macro,▁█
train_hamming_loss_macro,█▁
train_loss,█▁▁▁▁▁▁▁▁
val_auroc_macro,▁█
val_hamming_loss_macro,█▁
val_loss,█▁▁▁▁▁▁▁▁
test_auroc_macro,0.61805
test_hamming_loss_macro,0.21281
train_auroc_macro,0.64144


+-------+--------+--------+--------------------+-------------+
|  mode | #epoch |  loss  | hamming_loss_macro | auroc_macro |
+-------+--------+--------+--------------------+-------------+
| train |   0    | 69.485 |       0.6972       |    0.4988   |
| train |   1    | 1.0829 |         -          |      -      |
| train |   2    | 0.5451 |         -          |      -      |
| train |   3    | 0.5007 |         -          |      -      |
| train |   4    | 0.4844 |         -          |      -      |
| train |   5    | 0.4757 |         -          |      -      |
| train |   6    | 0.4681 |         -          |      -      |
| train |   7    | 0.4616 |         -          |      -      |
| train |   8    | 0.4558 |       0.2115       |    0.6414   |
+-------+--------+--------+--------------------+-------------+
+------+--------+--------+--------------------+-------------+----------------+
| mode | #epoch |  loss  | hamming_loss_macro | auroc_macro | early_stopping |
+------+--------+------

Epoch:0... Done
  Validating... Calculating val performance... Done
Done
Epoch:1... Done
  Validating... Done
Epoch:2... Done
  Validating... Done
Epoch:3... Done
  Validating... Done
Epoch:4... Done
  Validating... Done
Epoch:5... Done
  Validating... Done
-----------------------------EarlyStopping counter: 1 out of 10---------------------- best epoch currently 4
Epoch:6... Done
  Validating... Done
-----------------------------EarlyStopping counter: 2 out of 10---------------------- best epoch currently 4
Epoch:7... Done
  Validating... Done
-----------------------------EarlyStopping counter: 3 out of 10---------------------- best epoch currently 4
Epoch:8... Done
  Validating... Calculating val performance... Done
Done
-----------------------------EarlyStopping counter: 4 out of 10---------------------- best epoch currently 4
Starting testing... Calculating test performance... Done
Done


test_auroc_macro,▁
test_hamming_loss_macro,▁
train_auroc_macro,▁█
train_hamming_loss_macro,█▁
train_loss,█▂▁▁▁▁▁▁▁
val_auroc_macro,▁█
val_hamming_loss_macro,█▁
val_loss,█▁▁▁▁▁▁▁▁
test_auroc_macro,0.64255
test_hamming_loss_macro,0.20573
train_auroc_macro,0.80108


+-------+--------+---------+--------------------+-------------+
|  mode | #epoch |   loss  | hamming_loss_macro | auroc_macro |
+-------+--------+---------+--------------------+-------------+
| train |   0    | 20.8283 |       0.4194       |    0.5005   |
| train |   1    |  1.914  |         -          |      -      |
| train |   2    |  0.5461 |         -          |      -      |
| train |   3    |  0.5012 |         -          |      -      |
| train |   4    |  0.4607 |         -          |      -      |
| train |   5    |  0.419  |         -          |      -      |
| train |   6    |  0.3876 |         -          |      -      |
| train |   7    |  0.3623 |         -          |      -      |
| train |   8    |  0.3421 |       0.1426       |    0.8011   |
+-------+--------+---------+--------------------+-------------+
+------+--------+--------+--------------------+-------------+----------------+
| mode | #epoch |  loss  | hamming_loss_macro | auroc_macro | early_stopping |
+------+--